# Preprocessing III

This notebook extends the data output from preprocessing II, making it ready to train models on.

In [6]:
import pandas as pd
import numpy as np
import glob

path = './data/preprocessing_II'
files = glob.glob(path + "/*.csv")

## Import data

Data is read from preprocessing II output files. For a quick analysis of the data distribution, the fraction of patients suffering from kidney issues is printed.

In [7]:
def readFiles(csv_files):
    return [pd.read_csv(file) for file in csv_files]

ethn_list = readFiles(files)

Print ethnicity order and percentage of patients suffering from kidney disease

In [8]:
def printEthnicities(csv_files, df_list):
    for counter in range(len(csv_files)):
        print(counter.__str__() + "\t"  + (int(sum(df_list[counter]['has_kidney_issue'])/len(df_list[counter])*100)).__str__() + "% \t" + csv_files[counter].split("\\")[1].split(".")[0])

printEthnicities(files, ethn_list)

0	22% 	american_indian_alaska_native
1	11% 	asian
2	22% 	black_african_american
3	15% 	hispanic_latino
4	14% 	other
5	7% 	unable_to_obtain
6	14% 	unknown
7	18% 	white


In [9]:
def getTestDf(df_list):
    return df_list[0]

df = getTestDf(ethn_list)
df.head()

,Unnamed: 0,hadm_id,subject_id,admittime,admission_type,admission_location,insurance,marital_status,ethnicity,edregtime,...,emar_count,emar_charttime,emar_medications,emar_events,lab_count,lab_charttime,lab_flag,lab_priority,lab_comments,has_kidney_issue
0,0,20007016,11833347,2171-02-15 11:02:00,ELECTIVE,NaN,Other,NaN,AMERICAN INDIAN/ALASKA NATIVE,NaN,...,NaN,NaN,NaN,NaN,6.0,2171-02-16 23:22:00,"['', '', 'abnormal', '', '', '']","['STAT', 'STAT', 'STAT', 'STAT', 'STAT', 'STAT']","['', '', '', '', '', '']",False
1,1,20012443,17229742,2182-10-08 03:33:00,ELECTIVE,NaN,Other,NaN,AMERICAN INDIAN/ALASKA NATIVE,NaN,...,NaN,NaN,NaN,NaN,3.0,2182-10-10 07:57:00,"['', '', '']","['STAT', 'STAT', 'STAT']","['', '', '']",False
2,2,20016748,12255735,2147-05-08 20:52:00,EW EMER.,EMERGENCY ROOM,Other,MARRIED,AMERICAN INDIAN/ALASKA NATIVE,2147-05-08 16:23:00,...,NaN,NaN,NaN,NaN,98.0,2147-05-09 04:55:00,"['', '', 'abnormal', '', '', '', 'abnormal', '...","['ROUTINE', 'ROUTINE', 'ROUTINE', 'ROUTINE', '...","['', '', '', '', '', '', '', '', '', '', '', '...",True
3,3,20023656,15250287,2142-01-18 16:37:00,OBSERVATION ADMIT,PHYSICIAN REFERRAL,Other,SINGLE,AMERICAN INDIAN/ALASKA NATIVE,2142-01-18 13:48:00,...,49.0,2142-01-18 21:25:00,"['Heparin', 'LamoTRIgine', 'LevETIRAcetam', 'O...","['Not Given', 'Administered', 'Administered', ...",75.0,2142-01-19 16:45:00,"['', '', '', '', '', '', '', '', '', '', '', '...","['ROUTINE', 'ROUTINE', 'ROUTINE', 'ROUTINE', '...","['', '', '', '', '', '', '', '', '', '', '', '...",False
4,4,20025002,17923018,2167-11-03 03:27:00,ELECTIVE,NaN,Other,SINGLE,AMERICAN INDIAN/ALASKA NATIVE,NaN,...,NaN,NaN,NaN,NaN,102.0,2167-11-03 03:52:00,"['', '', '', '', '', 'abnormal', 'abnormal', '...","['', 'STAT', 'STAT', 'STAT', 'STAT', 'STAT', '...","['VEN. ', '', '', '', '', '', '', '', '', '',...",False


## Processing types of columns and categorical variables

The columns for 'emar_events', 'lab_priority' and 'lab_comments' are dropped as these do not contain interpretable data with predictive power.

In [10]:
datetime = ['admittime', 'edregtime', 'emar_charttime', 'lab_charttime']
ints = ['icd_code_count', 'emar_count', 'lab_count']
one_hots = ['admission_type', 'admission_location', 'insurance', 'marital_status', 'gender']
drop_cols = ['ethnicity', 'emar_events', 'lab_priority', 'lab_comments', 'lab_flag', 'Unnamed: 0', 'hadm_id', 'subject_id', 'edregtime', 'emar_charttime', 'lab_charttime']

def convert(type, dfs, cols):
    for df in dfs:
        for col in cols:
            df[col] = df[col].astype(type)
    return dfs

def findAllUniques(dfs, col):
    uniques = set()
    for df in dfs:
        for item in df[col].unique():
            uniques.add(item)
    return uniques

def oneHot(dfs, cols):
    for col in cols:
        vals = findAllUniques(dfs, col)
        for df in range(len(dfs)):
            if len(vals) > 2:
                for val in vals:
                    dfs[df][val] = np.where(dfs[df][col]==val, True, False)

                dfs[df] = dfs[df].drop(col, axis=1)
            else:
                dfs[df][col] = np.where(dfs[df][col]==val[0], True, False)
    return dfs

def dropCols(dfs, cols):
    for col in cols:
        for df in range(len(dfs)):
            dfs[df] = dfs[df].drop(col, axis=1)
    return dfs

ethn_list = convert('datetime64', ethn_list, datetime)
ethn_list = convert('Int64', ethn_list, ints)
ethn_list = oneHot(ethn_list, one_hots)
ethn_list = dropCols(ethn_list, drop_cols)

In [11]:
ethn_list[0].head()

,admittime,gender,anchor_age,anchor_year,icd_code_count,emar_count,emar_medications,lab_count,has_kidney_issue,EU OBSERVATION,...,PACU,EMERGENCY ROOM,INFORMATION NOT AVAILABLE,Medicaid,Other,Medicare,MARRIED,SINGLE,DIVORCED,WIDOWED
0,2171-02-15 11:02:00,False,0,2171,3,<NA>,NaN,6,False,False,...,False,False,False,False,True,False,False,False,False,False
1,2182-10-08 03:33:00,False,0,2182,3,<NA>,NaN,3,False,False,...,False,False,False,False,True,False,False,False,False,False
2,2147-05-08 20:52:00,False,60,2143,11,<NA>,NaN,98,True,False,...,False,True,False,False,True,False,True,False,False,False
3,2142-01-18 16:37:00,False,25,2142,8,49,"['Heparin', 'LamoTRIgine', 'LevETIRAcetam', 'O...",75,False,False,...,False,False,False,False,True,False,False,True,False,False
4,2167-11-03 03:27:00,False,0,2167,9,<NA>,NaN,102,False,False,...,False,False,False,False,True,False,False,True,False,False


## Process list variables

### emar_medication
This column is processed by finding the most common 5 medications among kidney disease patients.

These five are then one-hot encoded

In [12]:
def getLiterals(dfs, cols):
    chars = "[]''"
    for col in cols:
        for df in range(len(dfs)):
            dfs[df][col] = dfs[df][col].fillna("[]")
            for char in chars:
                dfs[df][col] = dfs[df][col].apply(lambda x: x.replace(char, ""))
            dfs[df][col] = dfs[df][col].apply(lambda x: x.split(", "))
    return dfs

def getUniquesInList(dfs, col):
    uniques = set()
    for df in range(len(dfs)):
        kid_iss = dfs[df][dfs[df]['has_kidney_issue'] == True]
        for item in kid_iss[col]:
                for sub in item:
                    uniques.add(sub)
    return uniques

def countMeds(dfs):
    uniques = getUniquesInList(dfs, 'emar_medications')
    meds = [0]*len(uniques)
    # for every medicine found in patients with kidney issues
    for i, med in enumerate(uniques):
        # for every ethnicity
        for df in range(len(dfs)):
            kid_iss = dfs[df][dfs[df]['has_kidney_issue'] == True]
            # for every patient's medications
            for item in kid_iss['emar_medications']:
                # if it contains the medicine, increment
                if med in item:
                    meds[i] += 1
    return meds

In [13]:
ethn_list = getLiterals(ethn_list, ['emar_medications'])
ethn_list[0].head()

,admittime,gender,anchor_age,anchor_year,icd_code_count,emar_count,emar_medications,lab_count,has_kidney_issue,EU OBSERVATION,...,PACU,EMERGENCY ROOM,INFORMATION NOT AVAILABLE,Medicaid,Other,Medicare,MARRIED,SINGLE,DIVORCED,WIDOWED
0,2171-02-15 11:02:00,False,0,2171,3,<NA>,[],6,False,False,...,False,False,False,False,True,False,False,False,False,False
1,2182-10-08 03:33:00,False,0,2182,3,<NA>,[],3,False,False,...,False,False,False,False,True,False,False,False,False,False
2,2147-05-08 20:52:00,False,60,2143,11,<NA>,[],98,True,False,...,False,True,False,False,True,False,True,False,False,False
3,2142-01-18 16:37:00,False,25,2142,8,49,"[Heparin, LamoTRIgine, LevETIRAcetam, OXcarbaz...",75,False,False,...,False,False,False,False,True,False,False,True,False,False
4,2167-11-03 03:27:00,False,0,2167,9,<NA>,[],102,False,False,...,False,False,False,False,True,False,False,True,False,False


In [14]:
kidney_medications = getUniquesInList(ethn_list, 'emar_medications')
kidney_medications = list(kidney_medications)

#med_count = countMeds(ethn_list)
#med_count = np.array(med_count)
# np.savetxt('med_frequency.csv', med_count, delimiter=",")
med_count = np.genfromtxt('med_frequency.csv', delimiter=",")



In [15]:
# N most frequent meds
N = 5

inds = np.argsort(med_count)
most_frequent = inds[-1]
most_frequent = np.append(most_frequent, inds[-3])
most_frequent = np.append(most_frequent, inds[-(N+2):-4])

freq_med_names = []

for item in most_frequent:
    freq_med_names.append(kidney_medications[item])


In [16]:
print(freq_med_names)

['Gas-X', 'Triamterene-Hydrochlorothiazide (37.5/25)', 'Fleet Enema', 'Midazolam Intradermal', 'Myrbetriq']


In [17]:
def oneHotMeds(dfs, meds):
    for med in meds:
        for df in range(len(dfs)):
            dfs[df][med] = False
            for row in dfs[df].itertuples():
                dfs[df].at[row.Index, med] = med in row.emar_medications
    dfs = dropCols(dfs, ['emar_medications'])
    return dfs

ethn_list = oneHotMeds(ethn_list, freq_med_names)
ethn_list[0].head()

,admittime,gender,anchor_age,anchor_year,icd_code_count,emar_count,lab_count,has_kidney_issue,EU OBSERVATION,ELECTIVE,...,Medicare,MARRIED,SINGLE,DIVORCED,WIDOWED,Gas-X,Triamterene-Hydrochlorothiazide (37.5/25),Fleet Enema,Midazolam Intradermal,Myrbetriq
0,2171-02-15 11:02:00,False,0,2171,3,<NA>,6,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,2182-10-08 03:33:00,False,0,2182,3,<NA>,3,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2147-05-08 20:52:00,False,60,2143,11,<NA>,98,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,2142-01-18 16:37:00,False,25,2142,8,49,75,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,2167-11-03 03:27:00,False,0,2167,9,<NA>,102,False,False,True,...,False,False,True,False,False,False,False,False,False,False


In [18]:
for df in ethn_list:
    df['admittime'] = pd.to_numeric(pd.to_datetime(df['admittime']))
    df['icd_code_count'] = df['icd_code_count'].fillna(0)
    #df['icd_code_count'] = df['icd_code_count'].astype(int)
    df['emar_count'] = df['emar_count'].fillna(0)
    # df['emar_count'] = df['emar_count'].astype(int)
    df['lab_count'] = df['lab_count'].fillna(0)

### Abnormal lab flags
The 'lab_flag' column is processed by counting the number of abnormal states

EDIT: lab_flag column is dropped as its information is already encoded as lab_count

In [19]:
def countAbnormal(dfs):
    for df in range(len(dfs)):
        for row in dfs[df].itertuples():
                print(row.lab_flag.count('abnormal'))
                # dfs[df].at[row.Index, 'lab_flag'] = row.lab_flag.count('abnormal')
    return dfs

In [20]:
ethn_list[0].head()

,admittime,gender,anchor_age,anchor_year,icd_code_count,emar_count,lab_count,has_kidney_issue,EU OBSERVATION,ELECTIVE,...,Medicare,MARRIED,SINGLE,DIVORCED,WIDOWED,Gas-X,Triamterene-Hydrochlorothiazide (37.5/25),Fleet Enema,Midazolam Intradermal,Myrbetriq
0,6346897320000000000,False,0,2171,3,0,6,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,6714329580000000000,False,0,2182,3,0,3,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,5596635120000000000,False,60,2143,11,0,98,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,5429349420000000000,False,25,2142,8,49,75,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,6243190020000000000,False,0,2167,9,0,102,False,False,True,...,False,False,True,False,False,False,False,False,False,False


In [21]:
df_name = ['american_indian_alaska_native', 'asian', 'black_african_american', 'hispanic_latino', 'other', 'unable_to_obtain', 'unknown', 'white']

for df in range(len(ethn_list)):
    print(df)
    ethn_list[df].to_csv("data/preprocessing_III/" + df_name[df] + ".csv")


0
1
2
3
4
5
6
7
